# TimeAndDate Weather Data

In [10]:
# Required libraries
import requests
from bs4 import BeautifulSoup
from datetime import datetime

In [11]:
# Adding the function to scrape weather data from TimeAndDate
def scrape_timeanddate(city, country):
    try:
        # Format the URL with city and country
        url = f"https://www.timeanddate.com/weather/{country}/{city}"
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for HTTP errors
        
        # Parse the content
        soup = BeautifulSoup(response.text, "html.parser")
        
        # Extract temperature
        temperature = soup.find('div', class_='h2').text.strip()
        
        # Extract weather condition
        condition = soup.find('div', class_='bk-focus__qlook').find('p').text.strip()

        # Extract details (visibility, pressure, humidity, dew point)
        details = soup.find_all('tr')
        visibility = details[6].find('td').text.strip()  # Adjust this index if necessary
        pressure = details[7].find('td').text.strip()
        humidity = details[8].find('td').text.strip()
        dew_point = details[9].find('td').text.strip()

        # Get the current time
        current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

        return {
            "city": city.title(),
            "temperature": temperature,
            "condition": condition,
            "current_time": current_time,
            "visibility": visibility,
            "pressure": pressure,
            "humidity": humidity,
            "dew_point": dew_point
        }

    except requests.exceptions.RequestException as e:
        print(f"Error fetching TimeAndDate data: {e}")
        return None

In [12]:
# Adding the function to save data to a text file
def save_weather_data(data, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        for key, value in data.items():
            f.write(f"{key.capitalize()}: {value}\n")
    print(f"Weather data saved to {filename}")

# Adding the main function to run the scraper
def main():
    city = input("Enter the city name: ").lower().replace(" ", "-")
    country = input("Enter the country name (use lowercase): ").lower()

    # Scrape from TimeAndDate
    timeanddate_data = scrape_timeanddate(city, country)
    if timeanddate_data:
        print("TimeAndDate Weather Data:")
        for key, value in timeanddate_data.items():
            print(f"{key.capitalize()}: {value}")
        save_weather_data(timeanddate_data, 'timeanddate_weather.txt')

# Run the main function
if __name__ == "__main__":
    main()

Enter the city name: Stockholm 
Enter the country name (use lowercase): sweden
TimeAndDate Weather Data:
City: Stockholm-
Temperature: 12 °C
Condition: Passing clouds.
Current_time: 2024-10-18 10:59:24
Visibility: 9 °C
Pressure: Now
Humidity: 
Dew_point: 12 °C
Weather data saved to timeanddate_weather.txt


# Wunderground Weather Data

In [4]:
# Required libraries
import requests
from bs4 import BeautifulSoup
from datetime import datetime

In [5]:
# Adding the function to scrape weather data from Wunderground

def scrape_wunderground(city, country):
    try:
        url = f"https://www.wunderground.com/{country}/{city}"
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for HTTP errors
        
        soup = BeautifulSoup(response.text, "html.parser")
        
        # Attempt to extract temperature
        temperature = soup.find('span', class_='wu-value wu-value-to')
        if not temperature:
            temperature = soup.find('span', class_='wu-value')
        
        temperature = temperature.text.strip() + ' °C' if temperature else "Temperature not available"
        
        # Extract weather condition
        condition = soup.find('div', class_='condition-icon small-6 medium-12 columns')
        condition = condition.text.strip() if condition else "Condition not available"

        # Extract additional details
        details_dict = {}
        weather_details = soup.find_all('span', class_='wu-value wu-value-to')
        
        for detail in weather_details:
            detail_id = detail.get('id')
            if detail_id:
                details_dict[detail_id] = detail.text.strip()

        visibility = details_dict.get('visibility', "Visibility not available")
        pressure = details_dict.get('pressure', "Pressure not available")
        humidity = details_dict.get('humidity', "Humidity not available")
        dew_point = details_dict.get('dewpoint', "Dew point not available")

        current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        
        return {
            "city": city.title(),
            "temperature": temperature,
            "condition": condition,
            "current_time": current_time,
            "visibility": visibility,
            "pressure": pressure,
            "humidity": humidity,
            "dew_point": dew_point
        }
    
    except requests.exceptions.RequestException as e:
        print(f"Error fetching Wunderground data: {e}")
        return None

def save_weather_data(data, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        for key, value in data.items():
            f.write(f"{key.capitalize()}: {value}\n")
    print(f"Weather data saved to {filename}")


In [6]:
def main():
    city = input("Enter the city name: ").lower().replace(" ", "-")
    country = input("Enter the country name (use lowercase): ").lower()
    
    wunderground_data = scrape_wunderground(city, country)
    if wunderground_data:
        print("Wunderground Weather Data:")
        for key, value in wunderground_data.items():
            print(f"{key.capitalize()}: {value}")
        save_weather_data(wunderground_data, 'wunderground_weather.txt')

if __name__ == "__main__":
    main()

Enter the city name: stockholm
Enter the country name (use lowercase): sweden
Error fetching Wunderground data: 404 Client Error: Not Found for url: https://www.wunderground.com/sweden/stockholm
